In [1]:
import pandas as pd
import numpy as np
import csv
from scipy.stats import norm
import os
from sqlalchemy import create_engine
import sqlalchemy as sqla
import pymysql


In [3]:
#PART 3

host = 'database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com'
username = 'user'
password = 'grad5100user'
port = 3306
dbname = 'topChef'
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{dbname}')


In [4]:
chefdetails = pd.read_sql_query("SELECT * FROM chefdetails",engine)
judges = pd.read_sql_query("SELECT * FROM judges",engine)


In [5]:
#Q1
q1chef = chefdetails[~chefdetails['season'].str.contains('Masters') & (chefdetails['season'] != 'Canada 6')]
q1judge = judges[~judges['season'].str.contains('Masters') & (judges['season'] != 'Canada 6')]

In [6]:
q1chef.head()

,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
0,0,Richard Blais,Richard B.,None,None,None,38.0,All Stars: New York,8,US,1,None,Owner,Male
1,1,Mike Isabella,Mike I.,None,None,None,35.0,All Stars: New York,8,US,2,None,Executive Chef/Owner,Male
2,2,Antonia Lofaso,Antonia L.,None,None,None,34.0,All Stars: New York,8,US,3,None,Executive Chef,Female
3,3,Tiffany Derry,Tiffany D.,None,None,None,27.0,All Stars: New York,8,US,4,POC,Executive Chef,Female
4,4,Carla Hall,Carla H.,None,None,None,46.0,All Stars: New York,8,US,5,POC,Chef/Owner,Female


In [7]:
q1judge.head()

,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
0,0,All Stars: New York,8,US,1,Quickfire,Team,Tom Colicchio,None,Top Chef Judge
1,1,All Stars: New York,8,US,1,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations
2,2,All Stars: New York,8,US,2,Quickfire,Individual,Joe Jonas,None,None
3,3,All Stars: New York,8,US,2,Elimination,Team,Katie Lee,None,Beat Bobby Flay Judge
4,4,All Stars: New York,8,US,3,Quickfire,Team,David Chang,None,None


In [8]:
#Q2

q2chef = chefdetails[~chefdetails['season'].str.contains('Masters') & (chefdetails['season'] != 'Canada 6')][['season', 'seasonNumber', 'name', 'placement', 'gender' ]]
q2judge = judges[~judges['season'].str.contains('Masters') & (judges['season'] != 'Canada 6')][['season', 'seasonNumber', 'episode', 'challengeType', 'guestJudge','competedOnTC' ]]

In [9]:
q2chef.head()


,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female


In [10]:
q2judge.head()

,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None


In [11]:
#Q3
q3chef = chefdetails[
    ~chefdetails['season'].str.contains('Masters') & 
    (chefdetails['season'] != 'Canada 6') & 
    chefdetails['seasonNumber'].between(1, 20)
][['season', 'seasonNumber']].drop_duplicates()

q3judge = chefdetails[
    ~chefdetails['season'].str.contains('Masters') & 
    (chefdetails['season'] != 'Canada 6') & 
    chefdetails['seasonNumber'].between(1, 20)
][['season', 'seasonNumber']].drop_duplicates()

In [12]:
q3chef.sort_values(by='seasonNumber')

,season,seasonNumber
295,San Francisco,1
190,Los Angeles,2
229,Miami,3
93,Chicago,4
263,New York,5
173,Las Vegas,6
109,D.C.,7
0,All Stars: New York,8
325,Texas,9
307,Seattle,10


In [14]:
q3judge.sort_values(by='seasonNumber')

,season,seasonNumber
295,San Francisco,1
190,Los Angeles,2
229,Miami,3
93,Chicago,4
263,New York,5
173,Las Vegas,6
109,D.C.,7
0,All Stars: New York,8
325,Texas,9
307,Seattle,10


In [15]:
#4a
q1chef['gender'].value_counts()

Male      170
Female    152
Name: gender, dtype: int64

In [16]:
#4b
winners = q1chef[q1chef['placement'] == 1]
winners['gender'].value_counts()

Male      14
Female     6
Name: gender, dtype: int64

In [17]:
#4c
top3 = q1chef[q1chef['placement'].isin([1, 2, 3])]
top3['gender'].value_counts()

Male      36
Female    25
Name: gender, dtype: int64

In [18]:
#4d
14/36 #men

0.3888888888888889

In [19]:
6/25 #women

0.24

In [20]:
#overall we can see that men who is top 3 have 38.9% chance to win the final champion, but women have only 24% to win. Women and men have similar people who joined the competition. The chance women won after placed top 3 is less than men but I think it seems just normal because people have different taste.

In [21]:
#Q5
guest_judge_status = winners['name'].apply(
    lambda name: 'Yes' if any(judges['guestJudge'] == name) else 'No'
)
guest_judge_status_df = pd.DataFrame(guest_judge_status).rename(columns={'name': 'GuestJudge'})

winner_judge_status = pd.concat([winners[['name', 'season']].reset_index(drop=True), guest_judge_status_df.reset_index(drop=True)], axis=1)

winner_judge_status.rename(columns={'name': 'Name', 'season': 'Season'}, inplace=True)

print(winner_judge_status)

                    Name               Season GuestJudge
0          Richard Blais  All Stars: New York        Yes
1           Melissa King       All-Stars L.A.        Yes
2                Mei Lin               Boston         No
3            Jeremy Ford           California         No
4      Brooke Williamson           Charleston        Yes
5        Stephanie Izard              Chicago        Yes
6           Kevin Sbraga                 D.C.         No
7           Joseph Flamm             Colorado         No
8             Kah-wai Lo              Houston         No
9   Kelsey Barnard Clark             Kentucky        Yes
10     Michael Voltaggio            Las Vegas        Yes
11             Ilan Hall          Los Angeles        Yes
12            Hung Huynh                Miami        Yes
13         Nicholas Elmi          New Orleans         No
14       Hosea Rosenberg             New York        Yes
15           Gabe Erales             Portland         No
16       Harold Dieterle       

In [22]:
#Q6
non_winners = chefdetails[chefdetails['placement'] != 1]

# Identify guest judges
guest_judges = judges[judges['competedOnTC'] == 'Yes']

# Find contestants who are in both groups and remove duplicates
non_winner_guest_judges = non_winners[non_winners['name'].isin(guest_judges['guestJudge'])].drop_duplicates(subset='name')

# Selecting only the name column
non_winner_guest_judges_names = non_winner_guest_judges[['name']]

# Display the names
print(non_winner_guest_judges_names)

                   name
1         Mike Isabella
2        Antonia Lofaso
3         Tiffany Derry
5            Dale Talde
13     Spike Mendelsohn
15     Stephen Asprinio
19      Bryan Voltaggio
22      Gregory Gourdet
24      Karen Akunowicz
25        Lee Anne Wong
27          Nini Nguyen
36         Melissa King
50         Amar Santana
54       Kwame Onwuachi
79       Sheldon Simeon
90           Sam Talbot
95        Richard Blais
127   Adrienne Cheatham
129        Carrie Baird
195     Michael Midgley
196        Betty Fraser
200  Josie Smith-Malave
245        Nina Compton
248       Carlos Gaytan
265      Stefan Richter
281        Dawn Burrell
282      Shota Nakajima
300      Miguel Morales
308   Brooke Williamson
329          Edward Lee
336    Nyesha Arrington
